In [1]:
import pysdds
import logging, io
import numpy as np

# To show logging messages in the notebook
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
# # Load simple 1-page file
# sdds = pysdds.read('../tests/files/sources/xLinac.matrix')
# 
# # Make it a bit more compact
# sdds.arrays = []
# sdds.columns = sdds.columns[:4]

In [3]:
sdds = pysdds.SDDSFile(add_data_nm=True)
sdds.set_mode('binary')

par_dbl = sdds.add_parameter('par1', 'double')
par_str = sdds.add_parameter('par2_str', 'string')

col_dbl = sdds.add_column('col1', 'double', units='m')
col_str = sdds.add_column('col2_str', 'string')
assert sdds.n_parameters == 2
assert sdds.n_columns == 2
print(sdds.describe())

SDDSFile: 0 pages, 2 parameters, 0 arrays, 2 columns, mode binary, endianness little
 Page sizes: []
 Column mem usage: 0.0000 MB


In [4]:
# Create a memory buffer for output - pysdds will automatically detect whether file or buffer is given
buf = io.BytesIO()

In [5]:
# begin streaming by creating a stream writer bound to the SDDS object with no data but correct structure (this is 
# already the case in our example)
sdds2 = sdds.copy(data=False)
sdds2.set_mode('binary')
stream_writer = sdds2.get_streaming_writer(filename=buf)

DEBUG:pysdds.writers.writers:Parameters: 2 of 2
DEBUG:pysdds.writers.writers:Parameter types: [dtype('float64'), None]
DEBUG:pysdds.writers.writers:Parameter lengths: [8, None]
DEBUG:pysdds.writers.writers:Columns: 2
DEBUG:pysdds.writers.writers:C types: [dtype('float64'), None]
DEBUG:pysdds.writers.writers:C lengths: [8, None]


In [6]:
# Write the header
stream_writer.begin()
print(buf.getvalue().decode('ascii'))

DEBUG:pysdds.writers.writers:Opening write stream to "<_io.BytesIO object at 0x000001A211C3A610>"
DEBUG:pysdds.writers.writers:Mode (binary), compression (None), endianness (little)
DEBUG:pysdds.writers.writers:Header write done in 0.074 ms


SDDS5
!# little-endian
!# fixed-rowcount
!Generated by pysdds 0.3.0+17.gb7792ee.dirty
!Submit issues at github.com/nikitakuklev/pysdds
&parameter name=par1, type=double,  &end
&parameter name=par2_str, type=string,  &end
&column name=col1, type=double, units=m,  &end
&column name=col2_str, type=string,  &end
&data mode=binary, endian=little, no_row_counts=0, column_major_order=0, &end


In [7]:
# start new page by providing parameter and/or array data as lists (must match the order in the file)
stream_writer.new_page(parameter_data=[0.05, 'bla'])

DEBUG:pysdds.writers.writers:Starting page 0 with 1000000 rows in FIXED COUNT MODE


In [8]:
print(buf.getvalue())

b'SDDS5\n!# little-endian\n!# fixed-rowcount\n!Generated by pysdds 0.3.0+17.gb7792ee.dirty\n!Submit issues at github.com/nikitakuklev/pysdds\n&parameter name=par1, type=double,  &end\n&parameter name=par2_str, type=string,  &end\n&column name=col1, type=double, units=m,  &end\n&column name=col2_str, type=string,  &end\n&data mode=binary, endian=little, no_row_counts=0, column_major_order=0, &end\n@B\x0f\x00\x9a\x99\x99\x99\x99\x99\xa9?\x03\x00\x00\x00bla'


In [9]:
stream_writer.write_rows(data_arrays=[1.0, 'test'])

DEBUG:pysdds.writers.writers:Single row [1.0, 'test'] converted to  [array([1.]), array(['test'], dtype='<U4')]
DEBUG:pysdds.writers.writers:Writing 1 rows to page 1
DEBUG:pysdds.writers.writers:Writing data [1.] with t='double' as view self.column_types[i]=dtype('float64')
DEBUG:pysdds.writers.writers:Writing data ['test'] with t='string' as string array


In [10]:
print(buf.getvalue())

b'SDDS5\n!# little-endian\n!# fixed-rowcount\n!Generated by pysdds 0.3.0+17.gb7792ee.dirty\n!Submit issues at github.com/nikitakuklev/pysdds\n&parameter name=par1, type=double,  &end\n&parameter name=par2_str, type=string,  &end\n&column name=col1, type=double, units=m,  &end\n&column name=col2_str, type=string,  &end\n&data mode=binary, endian=little, no_row_counts=0, column_major_order=0, &end\n@B\x0f\x00\x9a\x99\x99\x99\x99\x99\xa9?\x03\x00\x00\x00bla\x00\x00\x00\x00\x00\x00\xf0?\x04\x00\x00\x00test'


In [11]:
# it is more efficient to write multiple rows at once
stream_writer.write_rows(data_arrays=[np.array([1.1,1.5]),
                                      np.array(['foo','bar'])
                                      ])

DEBUG:pysdds.writers.writers:Writing 2 rows to page 1
DEBUG:pysdds.writers.writers:Writing data [1.1 1.5] with t='double' as view self.column_types[i]=dtype('float64')
DEBUG:pysdds.writers.writers:Writing data ['foo' 'bar'] with t='string' as string array


In [12]:
# Inspect resulting buffer
result = buf.getvalue()
print(result)

b'SDDS5\n!# little-endian\n!# fixed-rowcount\n!Generated by pysdds 0.3.0+17.gb7792ee.dirty\n!Submit issues at github.com/nikitakuklev/pysdds\n&parameter name=par1, type=double,  &end\n&parameter name=par2_str, type=string,  &end\n&column name=col1, type=double, units=m,  &end\n&column name=col2_str, type=string,  &end\n&data mode=binary, endian=little, no_row_counts=0, column_major_order=0, &end\n@B\x0f\x00\x9a\x99\x99\x99\x99\x99\xa9?\x03\x00\x00\x00bla\x00\x00\x00\x00\x00\x00\xf0?\x04\x00\x00\x00test\x9a\x99\x99\x99\x99\x99\xf1?\x03\x00\x00\x00foo\x00\x00\x00\x00\x00\x00\xf8?\x03\x00\x00\x00bar'


In [13]:
stream_writer.close()

DEBUG:pysdds.writers.writers:Ending page 1 implicitly due to close request
DEBUG:pysdds.writers.writers:Ending page 1
DEBUG:pysdds.writers.writers:Closing write stream


In [14]:
pysdds.readers.TRACE = True

In [15]:
sdds3 = pysdds.read(io.BufferedReader(io.BytesIO(result)))

DEBUG:pysdds.readers.readers:Opening file "<_io.BufferedReader>"
DEBUG:pysdds.readers.readers:Path opened as byte stream in 0.016 ms, size (None)
DEBUG:pysdds.readers.readers:Binary file endianness set to (little) from meta-command
DEBUG:pysdds.readers.readers:Fixed rowcount meta-command found - setting flag
DEBUG:pysdds.readers.readers:Binary file endianness set to (little) from data namelist
DEBUG:pysdds.readers.readers:Header parsed: 2 parameters, 0 arrays, 2 columns
DEBUG:pysdds.readers.readers:Parameters to parse: 2 of 2
DEBUG:pysdds.readers.readers:Parameter types: [dtype('float64'), None]
DEBUG:pysdds.readers.readers:Parameter lengths: [8, None]
DEBUG:pysdds.readers.readers:Columns to parse: 2
DEBUG:pysdds.readers.readers:Column types: [dtype('float64'), <class 'object'>]
DEBUG:pysdds.readers.readers:Column lengths: [8, None]
DEBUG:pysdds.readers.readers:All numeric: False
DEBUG:pysdds.readers.readers:Not all columns numeric, data is row order -> using slow sequential parser
DEB

In [17]:
sdds3.pdf()

,par1,par2_str
Page,,
0,0.05,bla


In [16]:
sdds3.cdf()

,col1,col2_str
0,1.0,test
1,1.1,foo
2,1.5,bar
